# NSTDemo Model Creation

This notebook should be able to run and output a .mlmodel for use with the Neural Style Transfer demo app **NSTDemo** ([project GitHub link](https://github.com/thesecretlab/ReinforceConf2019)). It requires pre-prepared folders of images as outlined in the project README.

Though it only executes a few simple command-line and python commands, it requires input to authorise Google Drive access (if required) and will take quite some time to run even on Google GPUs.

In [0]:
content_directory = 'training/content/'  # The directory of many varied images to train and test the model on styling
style_directory = 'training/style/'      # The directory of one or more images to create style models for
output_directory = 'training/test/'      # The directory to output models and test images to (will create if not exists)
max_iterations = 6000                    # The number of max_iterations to perform in training
export_shape = (800, 800)                # The dimensions of images the .mlmodel will accept to style 
output_model_name = 'StyleTransferModel' # The .model and .mlmodel filename to output when complete
testing_ratio = 0.1                      # Fraction of content images to use for model validation
data_uploaded = False                    # False if data in Google Drive, True if data uploaded to notebook Runtime directly

## Create Output Directory

In [0]:
  import os
  try:
    os.stat(output_directory)
  except:
    os.mkdir(output_directory)

## Import Training Data

Select one of the following methods.

### Navigate to Uploaded Training Data

This assumes you have uploaded a **training.zip** file containing the three directories of images using the upload files feature in the left sidebar. It should be still compressed, not contain any intermediate directories and unzip into the current working directory **/content**.



In [0]:
if data_uploaded:
  !unzip training.zip
  %cd training

### Navigate to  Training Data in Google Drive



---

**!!! Click the link and allow permissions when prompted !!!**

---

This assumes you have uploaded a folder called **training** into the root of your Google Drive, containing the three directories of images. It should be uncompressed and not contain any intermediate directories.

In [0]:
if not data_uploaded:
  from google.colab import drive
  drive.mount('/content/drive')
  %cd drive/My Drive

## Install and Adjust Dependencies

In [0]:
%%capture

# Remove CUDA 9, install CUDA 8
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/cuda-repo-ubuntu1604_8.0.61-1_amd64.deb
!dpkg -i --force-confnew cuda-repo-ubuntu1604_8.0.61-1_amd64.deb
!apt-get update
!apt-get install cuda-8-0

In [0]:
%%capture

# Force dpkg to overwrite the configuration file
!wget http://archive.ubuntu.com/ubuntu/pool/main/m/mesa/libglx-mesa0_18.0.5-0ubuntu0~18.04.1_amd64.deb
!dpkg -i --force-overwrite libglx-mesa0_18.0.5-0ubuntu0~18.04.1_amd64.deb
!wget http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/nvidia-410_410.48-0ubuntu1_amd64.deb
!dpkg -i --force-overwrite nvidia-410_410.48-0ubuntu1_amd64.deb
!apt --fix-broken install
!apt-get install cuda-8-0

In [0]:
%%capture

# Install TuriCreate
!pip install turicreate

# Remove MXNET for CUDA 9, install for CUDA 8
!pip uninstall -y mxnet
!pip install mxnet-cu80==1.1.0

## Create Model

### Perform Model Training

In [0]:
import turicreate as tc
  
# Load training images
styles = tc.load_images(style_directory)
training = tc.load_images(content_directory)

if testing_ratio > 0:
  testing, training = training.random_split(testing_ratio)

In [0]:
# Do the actual thing
model = tc.style_transfer.create(styles, training, max_iterations=max_iterations)

### Stylize Test Images

Creates a new directory at **training/test** and outputs stylized test images as .jpg files.

In [0]:
if testing_ratio > 0:
  for index, input in enumerate(testing.select_column('image')):
    output = model.stylize(input, style=index % styles.num_rows())
    output.save(output_directory +'/test-' + str(index) + '.jpg')

### Save and Export Model

In [0]:
model.save(output_directory +'/' +output_model_name + '.model')
model.export_coreml(output_directory +'/' +output_model_name +'.mlmodel')
model.export_coreml(output_directory +'/' +output_model_name + '.mlmodel', image_shape=export_shape)